In [3]:
from datetime import datetime

author = "johan-yapo"
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

# Create the author and timestamp markdown content
markdown_content = f"""
# Author and Timestamp Information

- **Author**: {author}
- **Date**: {timestamp}
"""

# Display the markdown content
from IPython.display import display, Markdown
display(Markdown(markdown_content))


# Author and Timestamp Information

- **Author**: johan-yapo
- **Date**: 2025-03-28 16:36:02


## Imports

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
from CBFV import composition
from time import time
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
#formula group needs to be integers
from sklearn.preprocessing import OrdinalEncoder
from matplotlib.patches import Patch
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import learning_curve
import pickle

import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default='browser'
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE 

ModuleNotFoundError: No module named 'numpy'

In [6]:
RNG_SEED = 8
np.random.seed(RNG_SEED)
curr_dir = os.getcwd()
DATA_PATH = os.path.join(os.path.dirname(curr_dir), 'Data')
MODELS_PATH = os.path.join(os.path.dirname(curr_dir), 'Models')

NameError: name 'np' is not defined